<a href="https://colab.research.google.com/github/madmjce2312/Energy/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Môi Trường

In [ ]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/PalmDetection

In [ ]:
# !unzip -q abc.zip

In [ ]:
ls

In [ ]:
!pip install mediapipe
# !pip install keras==2.3.1
# !pip install tensorflow==2.5.0

# Xử lý dữ liệu (tín hiệu số)

In [ ]:
import os
import cv2
import random
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
mp_draw = mp.solutions.drawing_utils
mp_hand = mp.solutions.hands

In [ ]:
hand = mp_hand.Hands(max_num_hands=1, min_detection_confidence=0.5)

In [ ]:
def get_lm(img, hand=hand):
    results = hand.process(img)
    if not results.multi_hand_landmarks:
        return None
    return results

In [ ]:
def preprocess(img, hand=hand, desiredHandWidth=221, desiredHandHeight=221):
    # BRG -> RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    results = get_lm(img)
    if results is None:
        return None
    
    # Flip if hand is upside down
    wrist = results.multi_hand_landmarks[0].landmark[0]
    if_mcp = results.multi_hand_landmarks[0].landmark[5]
    mf_mcp = results.multi_hand_landmarks[0].landmark[9]
    rf_mcp = results.multi_hand_landmarks[0].landmark[13]
    pf_mcp = results.multi_hand_landmarks[0].landmark[17]

    if all(y.y > wrist.y for y in [if_mcp, mf_mcp, rf_mcp, pf_mcp]):
        img = cv2.flip(img, 0)

    # Rotate to straight-up
    results = get_lm(img)
    if results is None:
        return None
    
    wrist = results.multi_hand_landmarks[0].landmark[0]
    if_mcp = results.multi_hand_landmarks[0].landmark[5]
    mf_mcp = results.multi_hand_landmarks[0].landmark[9]
    rf_mcp = results.multi_hand_landmarks[0].landmark[13]
    pf_mcp = results.multi_hand_landmarks[0].landmark[17]

    if all(x.x > wrist.x for x in [if_mcp, mf_mcp, rf_mcp, pf_mcp]):
        img = cv2.rotate(img, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)

    if all(x.x < wrist.x for x in [if_mcp, mf_mcp, rf_mcp, pf_mcp]):
        img = cv2.rotate(img, cv2.cv2.ROTATE_90_CLOCKWISE)
    
    # Flip if to left handed
    results = get_lm(img)
    if results is None:
        return None
    
    tf_mcp = results.multi_hand_landmarks[0].landmark[2]
    pf_mcp = results.multi_hand_landmarks[0].landmark[17]

    if tf_mcp.x > pf_mcp.x:
        img = cv2.flip(img, 1)
    
    # Align hand
    results = get_lm(img)
    if results is None:
        return None

    if_mcp = results.multi_hand_landmarks[0].landmark[5]
    pf_mcp = results.multi_hand_landmarks[0].landmark[17]

    if_mcp.x, if_mcp.y = if_mcp.x*w, if_mcp.y*h
    pf_mcp.x, pf_mcp.y = pf_mcp.x*w, pf_mcp.y*h

    root = ((if_mcp.x + pf_mcp.x)//2, (if_mcp.y + pf_mcp.y)//2)

    dY = pf_mcp.y - if_mcp.y
    dX = pf_mcp.x - if_mcp.x
    angle = np.degrees(np.arctan2(dY, dX))

    if abs(angle) > 90:
        angle -= 180

    dist = np.sqrt(dY**2 + dX**2)
    desiredDist = 0.8 * desiredHandWidth
    scale = desiredDist / dist

    M = cv2.getRotationMatrix2D(root, angle, scale)

    tX = desiredHandWidth * 0.5
    tY = desiredHandHeight * 0.3
    
    M[0,2] += tX - root[0]
    M[1,2] += tY - root[1]
    
    aligned = cv2.warpAffine(img, M, (desiredHandWidth, desiredHandHeight), flags=cv2.INTER_CUBIC)

    # Get palm
    gauss = cv2.GaussianBlur(aligned, (5,5), cv2.BORDER_DEFAULT)

    gray = cv2.cvtColor(gauss, cv2.COLOR_RGB2GRAY)

    eq_gray = cv2.equalizeHist(gray)

    # # histogram equalization rgb
    # img_yuv = cv2.cvtColor(aligned, cv2.COLOR_RGB2YUV)
    # img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    # img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

    canny = cv2.Canny(eq_gray, 100, 200)

    return canny

In [ ]:
def preprocess_and_save(input_path, output_path):
    for ppl in os.listdir(input_path):
        if not os.path.exists(f'{output_path}/{ppl}'):
            os.mkdir(f'{output_path}/{ppl}')
        print(ppl)
        for img_name in os.listdir(f'{input_path}/{ppl}'):
            print(img_name)
            img = cv2.imread(f'{input_path}/{ppl}/{img_name}')
            canny = preprocess(img)

            if canny is None:
                continue

            cv2.imwrite(f'{output_path}/{ppl}/{img_name}', canny)

In [ ]:
# preprocess real_db
preprocess_and_save(input_path='./data/Real_Db', output_path='./data/aligned')

# preprocess NotDb
preprocess_and_save(input_path='./data/NotDb', output_path='./data/aligned_NotDb')

# preprocess real_db
preprocess_and_save(input_path='./data/InDb', output_path='./data/aligned_InDb')

# Training feature extraction

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def convnet_model_():
    # vgg_model = keras.applications.VGG16(weights=None, include_top=False, input_shape=(221, 221, 3))
    # x = vgg_model.output
    cnn_input = Input(shape=(221, 221, 1))
    x = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(cnn_input)
    x = MaxPool2D(pool_size=(3,3), padding='same')(x)
    x = Conv2D(128, kernel_size=(3,3), padding='same', activation='relu')(x)
    x = MaxPool2D(pool_size=(3,3), padding='same')(x)
    x = Conv2D(256, kernel_size=(3,3), padding='same', activation='relu')(x)
    x = MaxPool2D(pool_size=(3,3), padding='same')(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = tf.nn.l2_normalize(x, axis=1)
    # x = Lambda(lambda x_: tf.math.l2_normalize(x,axis=1))(x)
#     x = Lambda(K.l2_normalize)(x)
    convnet_model = Model(inputs=cnn_input, outputs=x)
    return convnet_model

def deep_rank_model():
    convnet_model = convnet_model_()

    first_input = Input(shape=(221, 221, 1))
    first_conv = Conv2D(64, kernel_size=(3,3), strides=(16,16), padding='same', activation='relu')(first_input)
    first_max = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(first_conv)
    first_max = Flatten()(first_max)
    first_max = tf.nn.l2_normalize(first_max, axis=1)

    second_input = Input(shape=(221, 221, 1))
    second_conv = Conv2D(64, kernel_size=(3,3), strides=(32,32), padding='same', activation='relu')(second_input)
    second_max = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = tf.nn.l2_normalize(second_max, axis=1)
                       
    merge_one = concatenate([first_max, second_max])
    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(256)(merge_two)
    emb = Dense(128)(emb)
    l2_norm_final = tf.nn.l2_normalize(emb, axis=1)
                        
    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model

In [ ]:
# Xem cấu trúc model
from tensorflow.keras.utils import plot_model
model = deep_rank_model()
model.summary()
plot_model(model, to_file='model_feature_extraction.png')

In [ ]:
batch_size = 24
_EPSILON = 1e-7

def _loss_tensor(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, _EPSILON, 1.0 - _EPSILON)
    loss = 0.
    g = 1.
    for i in range(0, batch_size, 3):
        try:
            q_embedding = y_pred[i]
            p_embedding = y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p = tf.math.sqrt(tf.math.reduce_sum((q_embedding - p_embedding)**2))
            D_q_n = tf.math.sqrt(tf.math.reduce_sum((q_embedding - n_embedding)**2))
            loss = loss + g + D_q_p - D_q_n
        except:
            continue
    loss = loss/batch_size*3
    return tf.math.maximum(loss, 0)

In [ ]:
def image_generator(img_path, batch_size=24):
    identities = os.listdir(img_path)
    anchor_id = 0
    while True:
        input_queue = []

        for i in range(batch_size//3):

            idp = identities[anchor_id%len(identities)]
            idn = random.choice(identities)
            while idn == idp:
                idn = random.choice(identities)
            anchor_id += 1

            input_queue += [np.expand_dims(cv2.imread(f'{img_path}/{idp}/{img_name}', cv2.IMREAD_UNCHANGED)/255.0, 2) for img_name in random.sample(os.listdir(f'{img_path}/{idp}'), 2)]
            input_queue += [np.expand_dims(cv2.imread(f'{img_path}/{idn}/{img_name}', cv2.IMREAD_UNCHANGED)/255.0, 2) for img_name in random.sample(os.listdir(f'{img_path}/{idn}'), 1)]

        input_queue = np.array(input_queue)

        yield ([input_queue, input_queue, input_queue], np.zeros((batch_size, )))

In [ ]:
gen = image_generator('./data/aligned')

In [ ]:
l = next(gen)
print(l[0][0].shape)

In [ ]:
model.compile(loss=_loss_tensor, optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True))

In [ ]:
cb_es = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

In [ ]:
# Train 800 epochs without callback for firm performance
# Save model each 200 epochs
for i in range(4):
    model.fit_generator(generator=gen,
                    steps_per_epoch=300//batch_size,
                    epochs=200,
                    verbose=1)
    model.save_weights(f'./model/model64-{i}-2408.h5')

# Train 100 epochs with callback for best performance
model.fit_generator(generator=gen,
                   steps_per_epoch=360//batch_size,
                   epochs=100,
                   callbacks=[cb_es],
                   verbose=1)
model.save('./model/model128-best-2408.h5')

# Training classification

In [ ]:
md = deep_rank_model()
md.load_weights('./model/model128-best-2408.h5')

In [ ]:
# root_palm = {}
for person in os.listdir('./data/aligned'):
    if not os.path.exists(f'./data/emb/{person}'):
        os.mkdir(f'./data/emb/{person}')
    avg = 0
    count = 0
    for img_name in os.listdir(f'./data/aligned/{person}'):
        s1 = cv2.imread(f'./data/aligned/{person}/{img_name}', cv2.IMREAD_UNCHANGED)
        s1 = s1/255.0
        ip = np.expand_dims(s1, axis=2)
        ip = np.expand_dims(ip, axis=0)
        with open(f'./data/emb/{person}/{person}_{count}.txt', 'w+') as f:
            f.write(str([x for x in list(md.predict([ip, ip, ip])[0])]))
            f.close()
        
        # root_palm[person] = [str(x) for x in list(md.predict([ip, ip, ip])[0])]
        # break
    #     avg += md.predict([ip, ip, ip])[0]
        count += 1
    # avg = avg/count
    # root_palm[person] = [str(x) for x in list(avg)]

In [ ]:
import ast
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def get_all_emb(data_path='./data/emb'):
    X = []
    y = []
    classes = {p:i for i,p in enumerate(os.listdir('./data/emb'))}
    for person in os.listdir(data_path):
        for txt in os.listdir(os.path.join(data_path, person)):
            f = open(os.path.join(data_path, person, txt), 'r')
            temp = f.read()
            f.close()
            X.append(np.array(ast.literal_eval(temp)))
        y += [classes[person]]*len(os.listdir(os.path.join(data_path, person)))
      
    return np.array(X), np.array(y)

In [ ]:
X, y = get_all_emb()

In [ ]:
y = to_categorical(y, len(os.listdir('./data/emb')))

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=28)

In [ ]:
# Classification model
def init_dnn_classify(emb_size=128, num_classes=len(os.listdir('./data/emb'))):
    model = Sequential()
    model.add(Input(shape=(emb_size,)))
    # model.add(Dropout(0.2))
    # model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    # model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [ ]:
# Xem cấu trúc model
from tensorflow.keras.utils import plot_model
dnn = init_dnn_classify()
# dnn.summary()
plot_model(dnn, to_file='model_classification.png')

In [ ]:
dnn.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
dnn.fit(X, y, batch_size=16, epochs=100, validation_split=0.2)
dnn.save('./model/dnn_2408.h5')

# Testing

In [ ]:
# import json
# # write json root_palm
# with open('result.json', 'w') as fp:
#     json.dump(root_palm, fp)
#     fp.close()

In [ ]:
# # read json root_palm
# f = open('result.json', 'r')
# root_palm = {key:np.array([float(x) for x in value]) for key, value in json.load(f).items()}
# f.close()

In [ ]:
classes_inv = {i:p for i,p in enumerate(os.listdir('./data/emb'))}

In [ ]:
# Test InDb
for per in os.listdir('./data/InDb'):
    print("Ground truth: ", per)
    for img_name in os.listdir(f'./data/InDb/{per}'):
        img = cv2.imread(f'./data/InDb/{per}/{img_name}', cv2.IMREAD_UNCHANGED)
        img_canny = preprocess(img)
        if img_canny is not None:
            img_canny = img_canny/255.0
            ip = np.expand_dims(img_canny, axis=2)
            ip = np.expand_dims(ip, axis=0)

            emb = md.predict([ip, ip, ip])

            # result = min({per:np.linalg.norm(emb-val) for per,val in root_palm.items()}.items(), key = lambda x: x[1])

            prediction = dnn.predict(emb)[0]
            # print(prediction)
            predicted_person = classes_inv[np.argmax(prediction, 0)]

            if np.max(prediction, 0) < 0.5:
                print("Day la nguoi la")
            else:
                print(f"Day la {np.max(prediction, 0)} {predicted_person}")
        else:
            print("Cannot detect hand")

In [ ]:
# Test NotDb
for per in os.listdir('./data/NotDb'):
    print("Ground truth: ", per)
    for img_name in os.listdir(f'./data/NotDb/{per}'):
        img = cv2.imread(f'./data/NotDb/{per}/{img_name}', cv2.IMREAD_UNCHANGED)
        img_canny = preprocess(img)
        if img_canny is not None:
            img_canny = img_canny/255.0
            ip = img_canny
            ip = np.expand_dims(ip, axis=0)

            emb = md.predict([ip, ip, ip])

            # result = min({per:np.linalg.norm(emb-val) for per,val in root_palm.items()}.items(), key = lambda x: x[1])
            prediction = dnn.predict(emb)[0]
            # print(prediction)
            predicted_person = classes_inv[np.argmax(prediction, 0)]

            if np.max(prediction, 0) < 0.9:
                print("Day la nguoi la")
            else:
                print(f"Day la {np.max(prediction, 0)} {predicted_person}")
        else:
            print("Cannot detect hand")